# Code for Section 6.3.1 Validating Portfolio Optimization Solvers
The code below illustrates how you can validate your portfolio optimizers with various objectives:

1) target return
2) target risk
3) target risk and tracking error constraint
 
For the same combination of risk and returns targets, these solve equivalent problems. Hence, if you have a solution for one of them, you can use it to validate the others.

In [1]:
import numpy as np
import pandas as pd
import fortitudo.tech as ft
from scipy.linalg import cholesky
from cvxopt import matrix
from cvxopt.solvers import socp, options
from copy import copy

In [2]:
# Load multi-asset means and covariances
names, means, cov = ft.load_parameters()
I = len(names)

In [3]:
# Specify portfolio constraints (long-only, 25% upper bounds)
G = np.vstack((np.eye(I), -np.eye(I)))
h = np.hstack((0.25 * np.ones(I), np.zeros(I)))
return_target = 0.05

In [4]:
# Compute the base case with an expected return target
mv = ft.MeanVariance(means, cov, G, h)
e_star = mv.efficient_portfolio(return_target)

# Target risk optimization

In [5]:
# Use the volatility of e_star as a risk target
risk_target = np.sqrt(e_star.T @ cov @ e_star)[0, 0]
A = matrix(np.ones((1, I)))
b = matrix(np.array([1.]))

Gq = [matrix(np.zeros((I + 1, I)))]
hq = [matrix(np.zeros(I + 1))]
options['show_progress'] = False
scalar = 100
hq[0][0] = risk_target * scalar
L = -cholesky(scalar**2 * cov)
Gq[0][1:, 0:] = L
e_star_risk = np.array(socp(
    c=matrix(-means * scalar), Gl=matrix(G), hl=matrix(h), Gq=Gq, hq=hq, A=A, b=b)['x'])

In [6]:
# Validate the result
print(f'Max absolute exposure difference = {np.max(np.abs(e_star_risk - e_star))}')
print(f'Portfolio volatility = {np.sqrt(e_star_risk.T @ cov @ e_star_risk)[0, 0]} (target = {risk_target})')
print(f'Expected return = {(means @ e_star_risk)[0]}')

Max absolute exposure difference = 8.294811590370266e-08
Portfolio volatility = 0.06691865058412809 (target = 0.06691865030306862)
Expected return = 0.05000000024486122


# Target risk and tracking error constraint

In [7]:
# Compute target tracking error
e_bm = np.ones((I, 1)) / I
te_pf = e_star_risk - e_bm
te_target = np.sqrt(te_pf.T @ cov @ te_pf)[0, 0]

In [8]:
# Add benchmark relation and auxiliary variables
benchmark_relation = np.hstack((np.eye(I), -np.eye(I)))
A = matrix(np.vstack((np.hstack((np.ones(I), np.zeros(I))), benchmark_relation)))
b = matrix(np.vstack(([[1.]], e_bm)))
G = np.hstack((G, np.zeros((G.shape[0], I))))

In [9]:
# Add second-order cone constraints
Gq_base = matrix(np.zeros((I + 1, 2 * I)))
Gq2 = [copy(Gq_base), copy(Gq_base)]
Gq2[0][1:, 0:I] = L
Gq2[1][1:, I:] = L
hq2 = [matrix(np.zeros((I + 1))), matrix(np.zeros((I + 1)))]
hq2[0][0] = risk_target * scalar
hq2[1][0] = te_target * scalar

In [10]:
# Extend the expected return vector and optimize
means2 = np.hstack((means, np.zeros(I)))
e_star_dual_risk = np.array(socp(
    c=matrix(-means2 * scalar), Gl=matrix(G), hl=matrix(h), Gq=Gq2, hq=hq2, A=A, b=b)['x'][0:I])

In [11]:
# Validate the result
te_pf_opt = e_star_dual_risk - e_bm
print(f'Max absolute exposure difference = {np.max(np.abs(e_star_dual_risk - e_star_risk))}')
print(f'Portfolio volatility = {np.sqrt(e_star_dual_risk.T @ cov @ e_star_dual_risk)[0, 0]} (target = {risk_target})')
print(f'Tracking error volatility = {np.sqrt(te_pf_opt.T @ cov @ te_pf_opt)[0, 0]} (target = {te_target})')
print(f'Expected return = {(means @ e_star_dual_risk)[0]}')

Max absolute exposure difference = 1.4443682283538095e-07
Portfolio volatility = 0.06691864978535854 (target = 0.06691865030306862)
Tracking error volatility = 0.03180812083822808 (target = 0.031808128411388435)
Expected return = 0.05000000180651483


In [12]:
# Table 6.1 allocation overview
results = np.hstack((e_star, e_star_risk, e_star_dual_risk, e_bm))
pd.DataFrame(np.round(100 * results, 2),
             index=names,
             columns=['Target return', 'Target risk', 'Target risk and tracking error', 'Benchmark'])

,Target return,Target risk,Target risk and tracking error,Benchmark
Gov & MBS,0.00,0.00,0.00,10.0
Corp IG,0.00,-0.00,-0.00,10.0
Corp HY,0.00,0.00,0.00,10.0
EM Debt,18.39,18.39,18.39,10.0
DM Equity,0.00,0.00,0.00,10.0
EM Equity,0.00,-0.00,-0.00,10.0
Private Equity,6.61,6.61,6.61,10.0
Infrastructure,25.00,25.00,25.00,10.0
Real Estate,25.00,25.00,25.00,10.0
Hedge Funds,25.00,25.00,25.00,10.0
